# Aprendizaje automático relacional

#### Fernando Jesús Fernández Gallardo
#### Carmen Galván López

## Preparación

#### Imports y variables globales

In [ ]:
import pandas
import numpy as np
import sklearn
import networkx as nx
from sklearn import preprocessing, model_selection, naive_bayes, neighbors
from sklearn.model_selection import ShuffleSplit, cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from scipy import stats

semilla = 86

#### Lectura y procesamiento inicial de los datos brutos

In [ ]:
#Leemos los archivos
vertices = pandas.read_csv('data/political-books-nodes.csv')
aristas = pandas.read_csv('data/political-books-edges.csv')

#Borramos la columna ID
del(vertices['Id'])

#Mostramos las primeras 35 filas
vertices.head(35)

#### Selección y validación de los datos brutos

In [ ]:
"""
Comprobamos que el dataset es válido verificando que no existen duplicados
"""
if len(vertices) != len(set(vertices['Label'])):
    raise ValueError("El dataset no es válido ya que contiene duplicados")
"""
La mejor forma de identificar cada uno de los elementos que forma parte
del conjunto de entrenamiento es el nombre del propio libro (que en el dataset
se llama 'Label') en vez del ID o cualquier otro tipo de indentificador más
complejo. De esta forma, también es más fácil identificar elementos duplicados
(si los hubiera)
"""
atributos = vertices['Label']
"""
Nuestro objetivo es predecir la ideología política del autor basándonos en
sus obras, por lo que el objetivo que perseguimos en nuestro modelo
es el de la ideología política
"""
objetivos = vertices['political_ideology']

## Inicio del entrenamiento
#### Codificación del objetivo

In [ ]:

"""
Para poder trabajar con los datos que tenemos, necesitamos convertirlos en un formato que sklearn pueda "entender".
Debemos de hacer que nuestros datos "planos" sean para sklearn objetos "comparables", dependiendo del tipo de
ordenación que nosotros veamos más apropiada para el método en cuestión
(de una manera similar hacemos en Java cuando implementamos la interfaz 'Comparable' y el método compareTo)

El codificador adecuado para la variable objetivo es LabelEncoder, que trabaja
con una lista o array unidimensional de sus valores y admite cadenas

"""
# Codificadores
codificador_atributos = preprocessing.LabelEncoder()
codificador_objetivos = preprocessing.LabelEncoder()
# Datos codificados
atributos_codificados = codificador_atributos.fit_transform(atributos)
objetivos_codificados = codificador_objetivos.fit_transform(objetivos)

#### División en conjunto de entrenamiento y conjunto de prueba

Partimos el atributo y el objetivo en dos, de entrenamiento y de prueba

In [ ]:
(atributos_entrenamiento,
 atributos_prueba,
 objetivos_entrenamiento,
 objetivos_prueba) = model_selection.train_test_split(
        atributos_codificados,
        objetivos_codificados,
        # Valor de la semilla aleatoria para que el muestreo sea reproducible a pesar de ser aleatorio
        random_state=semilla,
        test_size=.33,
        stratify=objetivos_codificados
)

#Creamos nuevos ejemplos para futuras operaciones
nuevos_ejemplos = [[1.], [20.], [2.]]

cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)

## KNN no relacional

In [ ]:
def encontrar_mejor_k(atributos, objetivo, k_range, cv=5):
    puntajes_por_k = []

    # Convertir los atributos en un array bidimensional con una sola columna
    atributos = np.array(atributos).reshape(-1, 1)

    for k in k_range:
        # Crear clasificador KNN con el valor actual de k
        knn = KNeighborsClassifier(n_neighbors=k)

        # Suprimir los warnings relacionados con la siguiente versión de SciPy
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")

            # Realizar validación cruzada y obtener los puntajes
            puntajes = cross_val_score(knn, atributos, objetivo, cv=cv, scoring='accuracy')

        # Calcular el puntaje medio de validación cruzada
        puntaje_medio = puntajes.mean()

        # Almacenar el puntaje correspondiente al valor de k
        puntajes_por_k.append((k, puntaje_medio))

    return puntajes_por_k

encontrar_mejor_k(atributos_codificados, objetivos_codificados, k_range=list(range(1, 20)), cv=5)

In [ ]:
# Convertir los atributos y objetivos en arrays bidimensionales
atributos_entrenamiento_2d = np.array(atributos_entrenamiento).reshape(-1, 1)
atributos_prueba_2d = np.array(atributos_prueba).reshape(-1, 1)
objetivos_entrenamiento_2d = np.array(objetivos_entrenamiento).reshape(-1, 1)
objetivos_prueba_2d = np.array(objetivos_prueba).reshape(-1, 1)

#Según el k_scores, podemos ver que a partir de 10 vecinos es totalmente irrelevante cuantos pongamos, 
#así que utilizaremos n_neighbors=10,
#Definimos y entrenamos kNN

clasif_kNN = neighbors.KNeighborsClassifier(n_neighbors=10, metric='hamming')

clasif_kNN.fit(atributos_entrenamiento_2d, objetivos_entrenamiento)

# Probamos la predicción con los atributos de prueba
print('Predicción kNN:', codificador_objetivos.inverse_transform(clasif_kNN.predict(atributos_prueba_2d)))

# Hacemos el score con kNN
print('Precisión kNN:', clasif_kNN.score(atributos_prueba_2d, objetivos_prueba_2d))

# Hacemos el score con cross validation utilizando los datos de entrenamiento
cv_scores = cross_val_score(clasif_kNN, atributos_entrenamiento_2d, objetivos_entrenamiento, cv=cv)
print('Precisión cross validation:', np.mean(cv_scores))

In [ ]:
#Vamos a dar algunos datos sobre knn como pueden ser la distancia, los vecinos más cercanos a un dato, 
#y las clases de esos vecinos

distancias, vecinos = clasif_kNN.kneighbors(nuevos_ejemplos)
print("Primer ejemplo nuevo:", nuevos_ejemplos[0])
print("10 vecinos más cercanos:")
print([vecinos[0]])
print("Distancias a esos vecinos (cantidad de atributos con valores distintos / cantidad total de atributos):")
print(distancias[0])  
def obtenerClases(vecinos):
    l=[]
    for index in vecinos[0]:
        l.append(objetivos[index])
    return l
print("Clases a las que pertenecen esos vecinos: ", obtenerClases(vecinos))

## Sacar métricas relacionales

In [ ]:
# Cargar el grafo desde el archivo
df = pandas.read_csv('data/political-books-edges.csv')
grafo = nx.from_pandas_edgelist(df, 'Source', 'Target')

#### Degree centrality.

Métrica relacionada con la centralidad. El número de conexiones (enlaces) que tiene ese nodo. Los nodos con mayor grado se consideran más centrales en términos de conectividad.

In [ ]:
def obtenerDegreeCentrality(grafo):
    centrality = nx.degree_centrality(grafo)
    return list(centrality.values())

# Obtener la lista de Degree Centrality
degree_centrality = obtenerDegreeCentrality(grafo)

# Añadirlo a la tabla
vertices['Degree_Centrality'] = degree_centrality
vertices.head(25)

#### High closeness centrality.

Métrica relacionada con la centralidad. Si un nodo tiene un mayor grado, significa que está conectado a un mayor número de otros nodos en el grafo.

In [ ]:
def calcularHighClosenessCentrality(grafo):
    high_closeness = nx.closeness_centrality(grafo, u=None, distance=None, wf_improved=True)
    return list(high_closeness.values())

# Calcular el high closeness centrality
high_closeness = calcularHighClosenessCentrality(grafo)

# Añadirlo a la tabla
vertices['High_closeness_centrality'] = high_closeness
vertices.head(25)

#### High betweenness centrality.

Métrica relacionada con la centralidad. Un nodo con un alto valor de "high betweenness centrality" actúa como un puente o un punto de conexión crucial entre diferentes partes del grafo.

In [ ]:
def calcularHighBetweennessCentrality(grafo):
    betweenness = nx.betweenness_centrality(grafo, normalized=True, endpoints=False)
    return list(betweenness.values())

# Calcular el high betweenness centrality
high_betweenness = calcularHighBetweennessCentrality(grafo)

# Añadirlo a la tabla
vertices['High_betweenness_centrality'] = high_betweenness
vertices.head(25)

#### Coeficiente de clustering.

Es una medida que cuantifica qué tan conectados están los vecinos de un nodo en comparación con todas las posibles conexiones entre ellos.

In [ ]:
def obtenerCoeficienteClustering(grafo):
    # Obtener el coeficiente de clustering para cada vértice
    coeficientes = nx.clustering(grafo)
    return list(coeficientes.values())

# Obtener la lista de coeficientes de clustering
coeficientes_clustering = obtenerCoeficienteClustering(grafo)

# Añadirlo a la tabla
vertices['Clustering'] = coeficientes_clustering
vertices.head(25)

#### Modularidad.

Métrica relacionada con la detección de comunidades. Si un nodo tiene un mayor grado, significa que está conectado a un mayor número de otros nodos en el grafo.

In [ ]:
def calcularModularidad(grafo):
    particion = nx.community.greedy_modularity_communities(grafo)
    modularidad = nx.community.modularity(grafo, particion)
    return modularidad

# Calcular la modularidad
modularidad = calcularModularidad(grafo)

# Imprimir la modularidad
print(modularidad)

In [ ]:
G = nx.read_graphml('data/political-books-network.graphml')
nx.draw_random(G, with_labels=True)

## Selección de datos relacional

In [ ]:
#Dvisión de atributos y target
atributos = vertices[['Label', 'Degree_Centrality', 'High_closeness_centrality',
                     'High_betweenness_centrality','Clustering']]
objetivos = vertices['political_ideology']

#División de los datos en entreamiento y objetivo
(atributos_entrenamiento,
 atributos_prueba,
 objetivos_entrenamiento,
 objetivos_prueba) = model_selection.train_test_split(
        atributos_codificados,
        objetivos_codificados,
        # Valor de la semilla aleatoria para que el muestreo sea reproducible a pesar de ser aleatorio
        random_state=semilla,
        test_size=.33,
        stratify=objetivos_codificados
)

#Creamos nuevos ejemplos para futuras operaciones
nuevos_ejemplos = [[1., 0.23, 0.4321, 0.001, 0.333333], [20., 0.115, 0.3095, 0.00521, 0.5], [2., 0.0579, 0.3151, 0.0099, 0.6]]

In [ ]:
atributos_codificados = codificador_atributos.fit_transform(atributos)
objetivos_codificados = codificador_objetivos.fit_transform(objetivos)

## KNN relacional

In [ ]:
# Primero, vamos a ver con qué cantidad de vecinos tiene mayor score
# nuestro modelo gracias al uso de k-folds

encontrar_mejor_k(atributos_codificados, objetivos_codificados, k_range=list(range(1, 20)), cv=5)

In [ ]:
# Convertir los atributos y objetivos en arrays bidimensionales
atributos_entrenamiento_2d = np.array(atributos_entrenamiento).reshape(-1, 1)
atributos_prueba_2d = np.array(atributos_prueba).reshape(-1, 1)
objetivos_entrenamiento_2d = np.array(objetivos_entrenamiento).reshape(-1, 1)
objetivos_prueba_2d = np.array(objetivos_prueba).reshape(-1, 1)

#Según el k_scores, podemos ver que a partir de 10 vecinos es totalmente irrelevante cuantos pongamos, 
#así que utilizaremos n_neighbors=10,
#Definimos y entrenamos kNN

clasif_kNN = neighbors.KNeighborsClassifier(n_neighbors=10, metric='hamming')

clasif_kNN.fit(atributos_entrenamiento_2d, objetivos_entrenamiento)

# Probamos la predicción con los atributos de prueba
print('Predicción kNN:', clasif_kNN.predict(atributos_prueba_2d))

# Hacemos el score con kNN
print('Precisión kNN:', clasif_kNN.score(atributos_prueba_2d, objetivos_prueba_2d))

# Hacemos el score con cross validation utilizando los datos de entrenamiento
cv_scores = cross_val_score(clasif_kNN, atributos_entrenamiento_2d, objetivos_entrenamiento, cv=cv)
print('Precisión cross validation:', np.mean(cv_scores))

In [ ]:
#Vamos a dar algunos datos sobre knn como pueden ser la distancia, los vecinos más cercanos a un dato, 
#y las clases de esos vecinos

distancias, vecinos = clasif_kNN.kneighbors(nuevos_ejemplos)
print("Primer ejemplo nuevo:", nuevos_ejemplos[0])
print("10 vecinos más cercanos:")
print([vecinos[0]])
print("Distancias a esos vecinos (cantidad de atributos con valores distintos / cantidad total de atributos):")
print(distancias[0])
print("Clases a las que pertenecen esos vecinos: ", obtenerClases(vecinos))